<a href="https://colab.research.google.com/github/mars-ef/Trustpilot_Scraper/blob/main/Trustpilot_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Analytics with Adam video on how to use https://www.youtube.com/watch?v=VD0wIiLVSRA
#Original code: https://github.com/analyticswithadam/Python/blob/main/Web_Scraping_from_TrustPilot.ipynb

from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def extract_data(element, page_number):
    """Extract relevant data from a review element."""
    reply_date = element.select_one('.styles_replyInfo__FYSje time')
    return {
        'brand': "Chase", #Change to your brand
        'username': element.select_one('[data-consumer-name-typography]').text,
        'location': element.select_one('[data-consumer-country-typography] span').text,
        'number_of_reviews': element.select_one('[data-consumer-reviews-count]').text.split(' ')[0] if element.select_one('[data-consumer-reviews-count]') else None,
        'review_rating': element.select_one('[data-service-review-rating] img')['alt'].split(' ')[1],
        'date': element.select_one('[data-service-review-rating] time')['datetime'].split('T')[0],
        'invited': element.select_one('[data-review-label-tooltip-trigger] span').text if element.select_one('[data-review-label-tooltip-trigger] span') else None,
        'review_title': element.h2.text,
        'review_text': element.select_one('[data-service-review-text-typography]').text if element.select_one('[data-service-review-text-typography]') else None,
        'date_of_experience': pd.to_datetime(element.select_one('[data-service-review-date-of-experience-typography]').text.split(': ')[-1]),
        'reply_from_business': element.select_one('[data-service-review-business-reply-text-typography]').text if element.select_one('[data-service-review-business-reply-text-typography]') else None,
        'reply_date': reply_date['datetime'].split('T')[0] if reply_date else None,
        'page_number': page_number
    }

def scrape_reviews(company, from_page=1, to_page=1):
    """Scrape reviews from Trustpilot for a given company."""
    data = []
    for i in range(from_page, to_page + 1):
        response = requests.get(f"https://www.trustpilot.com/review/{company}?page={i}") #for most recent change to '{company}page={i}&sort=recency'
        soup = BeautifulSoup(response.content, 'html.parser')

        for element in soup.select('article'):
            data.append(extract_data(element, i))

        sleep(1)  # To prevent throttling

    return pd.DataFrame(data)

# Choose brand and page range
company = 'chase.co.uk' #E.g. for https://www.trustpilot.com/review/wise.com to sort default take 'wise.com'.
review_data = scrape_reviews(company, 1, 3)

In [2]:
review_data

,brand,username,location,number_of_reviews,review_rating,date,invited,review_title,review_text,date_of_experience,reply_from_business,reply_date,page_number
0,Chase,Cynthia,NL,19,5,2024-10-24,Redirected,Excellent for using abroad,Excellent for using abroad. Great to see exact...,2024-10-24,None,None,1
1,Chase,Andy B.,GB,32,1,2024-10-23,None,When I contacted Chse by phone to hear…,When I contacted Chse by phone to hear a lovel...,2024-10-22,None,None,1
2,Chase,richard hardaker,GB,1,4,2024-10-24,Redirected,Convenient with good interest paid,"I find the service provided very convenient, a...",2024-10-24,None,None,1
3,Chase,Chris,GB,1,5,2024-10-24,Redirected,The app has been the best I've ever…,"The app has been the best I've ever used, and ...",2024-10-24,None,None,1
4,Chase,Helena,GB,10,5,2024-10-23,None,Give Cashback on current account,"Give Cashback on current account, saver accoun...",2024-10-23,None,None,1
5,Chase,Arron A Small,GB,6,5,2024-10-23,None,Lighting telephone centre staff!,I’m in the process of purchasing property numb...,2024-10-22,None,None,1
6,Chase,Lau Berraondo,GB,1,5,2024-10-24,Redirected,Overall simple and excellent,Generally a great back with an excellent APP. ...,2024-10-24,None,None,1
7,Chase,Bryan Levy,GB,47,4,2024-10-24,Redirected,A fast friendly efficient banking service,I've had an account with Chase since they star...,2024-10-24,None,None,1
8,Chase,Consumer,GB,4,5,2024-10-24,Redirected,Great online banking,I have found Chase to be excellent. The intere...,2024-10-24,None,None,1
9,Chase,MrsM,GB,5,5,2024-10-23,Redirected,Brilliant bank,"I love the cashback, the easy, straightforward...",2024-10-23,None,None,1


In [ ]:
# Convert the DataFrame to a CSV file and download it
review_data.to_csv('reviews.csv', index=False)
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>